In [50]:
import dataset 
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import time

In [23]:
from  numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
print(tf.__version__)
tf.get_default_graph()

1.8.0


load du lieu

In [24]:
# khoi tao gia tri load du lieu


validation_size = 0.2
test_size = 0.3
img_size = 64
num_input_channels = 3
train_path=r'D:\Hoang\data_use_kl\data_use_kl'
classes = dataset.get_labels(train_path)

print(classes)

num_classes = len(classes)
print(num_classes)

['a_h', 'a_k', 'chi_h', 'chi_k', 'e_h', 'e_k', 'fu_h', 'fu_k', 'ha_h', 'ha_k', 'he_h', 'he_k', 'hi_h', 'hi_k', 'ho_h', 'ho_k', 'i_h', 'i_k', 'ka_h', 'ka_k', 'ke_h', 'ke_k', 'ki_h', 'ki_k', 'ko_h', 'ko_k', 'ku_h', 'ku_k', 'ma_h', 'ma_k', 'me_h', 'me_k', 'mi_h', 'mi_k', 'mo_h', 'mo_k', 'mu_h', 'mu_k', 'na_h', 'na_k', 'ne_h', 'ne_k', 'ni_h', 'ni_k', 'no_h', 'no_k', 'nu_h', 'nu_k', 'n_h', 'n_k', 'o_h', 'o_k', 'ra_h', 'ra_k', 're_h', 're_k', 'ri_h', 'ri_k', 'ro_h', 'ro_k', 'ru_h', 'ru_k', 'sa_h', 'sa_k', 'se_h', 'se_k', 'shi_h', 'shi_k', 'so_h', 'so_k', 'su_h', 'su_k', 'ta_h', 'ta_k', 'te_h', 'te_k', 'to_h', 'to_k', 'tsu_h', 'tsu_k', 'u_h', 'u_k', 'wa_h', 'wa_k', 'wo_h', 'wo_k', 'ya_h', 'ya_k', 'yo_h', 'yo_k', 'yu_h', 'yu_k']
92


In [25]:
#load du lieu 
data=dataset.read_train_sets(train_path=train_path,image_size=img_size,test_size=test_size,validation_size=validation_size)
print(data.train.images.shape)
print(data.valid.images.shape)
print(data.test.images.shape)

(51629, 64, 64, 3)
(20651, 64, 64, 3)
(30976, 64, 64, 3)


In [43]:
data.train.images.shape[0]

51629

# Xây dựng hàm cho CNN

Khởi tạo giá trị

In [34]:
tf.reset_default_graph()

with tf.name_scope("inputs"):
    x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_input_channels], name='x')

    # labels
    y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

    keep_prob=tf.constant(1.0,dtype=tf.float32,name="keep_prob")
    
    y_true_cls = tf.argmax(y_true, axis=1)

In [35]:
# cau truc CNN M6-1

filter_size_conv1 = 3 
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64

filter_size_conv4 = 3
num_filters_conv4 = 64
    
fc_layer_size = 256

In [36]:
def create_weights(shape,name):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05),name=name)

def create_biases(size,name):
    return tf.Variable(tf.constant(0.05, shape=[size]),name=name)

In [37]:
def create_conv_layer(input,
               num_input_channels, # so channels của từng ảnh
               filter_size,    # kích thước filter, kenel, 
               num_filters,  # số lượng đầu ra cho layer sau
               use_maxpool=False, #sử dụng max_pooling k?
               name = "conv_x" ):  
    with tf.name_scope(name):
        #tao weight và bias cho lop convolutional nay/
#         print("input {}: {}".format(name,input))

        name_weights = name+"_weights"
        with tf.name_scope("weights"):
            weights = create_weights(shape=[filter_size, filter_size, num_input_channels, num_filters],name=name_weights)
#         print("weights {}: {}".format(name,weights))

        name_biases = name+"_biases"
        with tf.name_scope("biases"):
            biases = create_biases(num_filters,name=name_biases)
        ## xay dung convolution layer, layer trả về có dimension nhu input
#         with tf.device('/gpu:0'):
        layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')
#         print("layer {}: {}".format(name,layer))

        layer += biases

        layer=tf.nn.dropout(layer,keep_prob);

        layer = tf.nn.relu(layer)

        if use_maxpool:
            layer = tf.nn.max_pool(value=layer,
                                    ksize=[1, 2, 2, 1],
                                    strides=[1, 2, 2, 1],
                                    padding='SAME')


#         store in histogram
        tf.summary.histogram("weights",weights)
        tf.summary.histogram("biases",biases)
        tf.summary.histogram("layer",layer)
        
        
        layer = tf.identity(layer,name=name)
        return layer

def create_flatten_layer(layer, name="flatten"):
    with tf.name_scope(name):
        # chuyen mỗi ảnh thanh mot vector, moi anh sẽ có kích thước phẳng là: img_size * img_size *num_channels

        #dimension layer: batch_size,img_size,img_size,num_channels 

        #lấy số lượng data
        layer_shape = layer.get_shape()

        # số lượng feature mỗi ảnh = img_size * img_size *num_channels
        num_features = layer_shape[1:4].num_elements()

        # reshape lại data ví dụ (10,2,2,5) // có 10 ảnh và mỗi ảnh có 2*2*5=20 feature => flat_layer: (10,20) 
        layer = tf.reshape(layer, [-1, num_features])
#         print("layer {}: {}".format(name,layer))
        layer = tf.identity(layer,name=name)
        return layer

def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=False, 
             use_dropout=False,
             name = "fc"):
    
    with tf.name_scope(name):
        name_weights = name+"_weights"
        name_biases = name+"_biases"
        with tf.name_scope("weights"):
            weights = create_weights(shape=[num_inputs, num_outputs],name=name_weights)
        with tf.name_scope("biases"):
            biases = create_biases(num_outputs,name=name_biases)
    
#         with tf.device('/gpu:0'):
        layer = tf.matmul(input, weights) + biases

        if use_relu:
            layer = tf.nn.relu(layer)
        if use_dropout:
             layer=tf.nn.dropout(layer,keep_prob);
#         print("layer {}: {}".format(name,layer))

        tf.summary.histogram("weights",weights)
        tf.summary.histogram("biases",biases)
        tf.summary.histogram("layer",layer)
        
        layer = tf.identity(layer,name=name)
        return layer

# Xây dựng cấu trúc CNN
conv3-32 -> conv3-32 ->maxpool -> conv3-64 -> conv3-64 -> maxpool ->fc-256

In [38]:
conv1 =  create_conv_layer(input=x,num_input_channels=num_input_channels,
                           filter_size=filter_size_conv1,num_filters=num_filters_conv1,use_maxpool=False, name="conv1")

conv2 = create_conv_layer(input=conv1,num_input_channels=num_filters_conv1,
                          filter_size=filter_size_conv2,num_filters=num_filters_conv2,use_maxpool=True, name="conv2")

conv3 = create_conv_layer(input=conv2,num_input_channels=num_filters_conv2,
                          filter_size=filter_size_conv3,num_filters=num_filters_conv3,use_maxpool=False, name="conv3")

conv4 = create_conv_layer(input=conv3,num_input_channels=num_filters_conv3,
                          filter_size=filter_size_conv4,num_filters=num_filters_conv4,use_maxpool=True, name="conv4")

layer_flat = create_flatten_layer(conv4)

num_feature = layer_flat.get_shape()[1:2].num_elements()

fc1 = create_fc_layer(input=layer_flat,num_inputs=num_feature,num_outputs=fc_layer_size,use_relu=True,use_dropout=True,name="fc1")


fc2 = create_fc_layer(input=fc1,num_inputs=fc_layer_size,num_outputs=num_classes,use_relu=False,use_dropout=True,name="fc2")



predict

In [39]:
with tf.name_scope("Train"):
#     with tf.device('/gpu:0'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc2,
                                                        labels=y_true)
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(cross_entropy)
        tf.summary.scalar("loss",loss)
    with tf.name_scope("train_step"):
        train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)


with tf.name_scope("Test"):
    y_pred = tf.nn.softmax(fc2,name='y_pred')
    y_pred_cls = tf.argmax(y_pred, axis=1)
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")



In [51]:
my_config =tf.ConfigProto(allow_soft_placement=True,log_device_placement=True)
my_config.gpu_options.allow_growth = True
session = tf.Session(config=my_config)

session.run(tf.global_variables_initializer())

saver = tf.train.Saver()

merged =tf.summary.merge_all();
train_writer= tf.summary.FileWriter(r".\graph\M6\train",session.graph)
valid_writer=tf.summary.FileWriter(r".\graph\M6\valid",session.graph)


batch_size=100
def train(num_epochs):
    num_iteration = int(data.train.images.shape[0]/batch_size)
    total_iterations =0
    for epoch in range(num_epochs):
        for i in range(num_iteration):

            x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
    #         print(y_true_batch.shape)
            feed_dict_train = {x: x_batch, y_true: y_true_batch}    

            session.run(train_step, feed_dict=feed_dict_train)

            if i % 100== 0 and i>0: 
                x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch_size)

                feed_dict_valid = {x: x_valid_batch, y_true: y_valid_batch }

                valid_loss,valid_accuracy,valid_merged = session.run([loss,accuracy,merged], feed_dict=feed_dict_valid)

                feed_dict_train = {x: x_batch, y_true: y_true_batch} 
                train_accuracy,train_merged = session.run([accuracy,merged], feed_dict=feed_dict_train)
                
                total_iterations=total_iterations+100

                msg="epoch: {0:d} -- iteration: {1:d} --- loss: {2:.2f} --- train_acc {3:>6.2%} --- valid_acc {4:6.2%}"

                print(msg.format(epoch,total_iterations,valid_loss,train_accuracy,valid_accuracy))

                print()

                train_writer.add_summary(train_merged,i)
                valid_writer.add_summary(valid_merged,i)
            
            
        save_path=saver.save(session, r'.\model\modelxxx.ckpt',global_step=epoch)
        

In [53]:
start_time = time.time()
print(f"start time: {start_time}")
train(num_epochs=21)
end_time= time.time()
print(f"end time: {end_time}")
run_time =end_time-start_time
print(f"run time: {run_time}")

start time: 1526485817.1148167
epoch: 0 -- iteration: 100 --- loss: 3.81 --- train_acc  6.00% --- valid_acc 12.00%

epoch: 0 -- iteration: 200 --- loss: 2.95 --- train_acc 38.00% --- valid_acc 28.00%

epoch: 0 -- iteration: 300 --- loss: 2.41 --- train_acc 53.00% --- valid_acc 42.00%

epoch: 0 -- iteration: 400 --- loss: 2.26 --- train_acc 53.00% --- valid_acc 41.00%

epoch: 0 -- iteration: 500 --- loss: 2.05 --- train_acc 48.00% --- valid_acc 51.00%

epoch: 1 -- iteration: 600 --- loss: 1.48 --- train_acc 68.00% --- valid_acc 61.00%

epoch: 1 -- iteration: 700 --- loss: 1.72 --- train_acc 72.00% --- valid_acc 57.00%

epoch: 1 -- iteration: 800 --- loss: 1.23 --- train_acc 70.00% --- valid_acc 71.00%

epoch: 1 -- iteration: 900 --- loss: 0.95 --- train_acc 73.00% --- valid_acc 75.00%

epoch: 1 -- iteration: 1000 --- loss: 0.89 --- train_acc 73.00% --- valid_acc 73.00%

epoch: 2 -- iteration: 1100 --- loss: 1.11 --- train_acc 82.00% --- valid_acc 70.00%

epoch: 2 -- iteration: 1200 --- 

epoch: 19 -- iteration: 9600 --- loss: 0.59 --- train_acc 100.00% --- valid_acc 90.00%

epoch: 19 -- iteration: 9700 --- loss: 0.69 --- train_acc 100.00% --- valid_acc 90.00%

epoch: 19 -- iteration: 9800 --- loss: 1.00 --- train_acc 100.00% --- valid_acc 92.00%

epoch: 19 -- iteration: 9900 --- loss: 0.20 --- train_acc 100.00% --- valid_acc 95.00%

epoch: 19 -- iteration: 10000 --- loss: 0.49 --- train_acc 100.00% --- valid_acc 94.00%

epoch: 20 -- iteration: 10100 --- loss: 0.11 --- train_acc 100.00% --- valid_acc 96.00%

epoch: 20 -- iteration: 10200 --- loss: 0.23 --- train_acc 100.00% --- valid_acc 95.00%

epoch: 20 -- iteration: 10300 --- loss: 0.72 --- train_acc 99.00% --- valid_acc 93.00%

epoch: 20 -- iteration: 10400 --- loss: 0.36 --- train_acc 100.00% --- valid_acc 89.00%

epoch: 20 -- iteration: 10500 --- loss: 0.32 --- train_acc 100.00% --- valid_acc 94.00%

end time: 1526487187.622616
run time: 1370.5077993869781


In [54]:
run_time/60

22.841796656449635

In [20]:
a=1235
b=0.2312
c=0.389444
d=0.56876525
msg="train_iteration {0:d} --- loss: {1:.2f} --- train_acc {2:>6.2%} --- valid_acc {3:6.2%}"
print(msg.format(a,b,c,d))

train_iteration 1235 --- loss: 0.23 --- train_acc 38.94% --- valid_acc 56.88%


In [32]:

for i in range(10):
    for j in range(5):
        print(j+10)

10
11
12
13
14
10
11
12
13
14
10
11
12
13
14
10
11
12
13
14
10
11
12
13
14
10
11
12
13
14
10
11
12
13
14
10
11
12
13
14
10
11
12
13
14
10
11
12
13
14
